In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import carla #the sim library itself
import time # to set a delay after each photo
import cv2 #to work with images from cameras
import numpy as np #in this example to change image representation - re-shaping
import math
import sys
import random
sys.path.append('C:/CARLA_0.9.13/PythonAPI/carla') # tweak to where you put carla
from tensorflow.keras.models import load_model
from agents.navigation.global_route_planner import GlobalRoutePlanner
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms


PREFERRED_SPEED = 30
SPEED_THRESHOLD = 2

MAX_STEER_DEGREES = 40
STEERING_CONVERSION = 75

CAMERA_POS_Z = 1.3
CAMERA_POS_X = 1.4

HEIGHT = 180
WIDTH = 320

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# utility function for camera listening
def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))[:, :, :3]


def sem_callback(image,data_dict):
    ########## IMPORTANT CHANGE for Semantic camera ##############
    image.convert(carla.ColorConverter.CityScapesPalette)
    data_dict['sem_image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))[:, :, :3]

# maintain speed function
def maintain_speed(s):
    '''
    this is a very simple function to maintan desired speed
    s arg is actual current speed
    '''
    if s >= PREFERRED_SPEED:
        return 0
    elif s < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.9 # think of it as % of "full gas"
    else:
        return 0.4 # tweak this if the car is way over or under preferred speed


# function to get angle between the car and target waypoint
def get_angle(car,wp):
    vehicle_pos = car.get_transform()
    car_x = vehicle_pos.location.x
    car_y = vehicle_pos.location.y
    wp_x = wp.transform.location.x
    wp_y = wp.transform.location.y

    # vector to waypoint
    x = (wp_x - car_x)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    y = (wp_y - car_y)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5

    #car vector
    car_vector = vehicle_pos.get_forward_vector()
    degrees = math.degrees(np.arctan2(y, x) - np.arctan2(car_vector.y, car_vector.x))
    # extra checks on predicted angle when values close to 360 degrees are returned
    if degrees<-180:
        degrees = degrees + 360
    elif degrees > 180:
        degrees = degrees - 360
    return degrees

def get_proper_angle(car,wp_idx,rte):
    # create a list of angles to next 5 waypoints starting with current
    next_angle_list = []
    for i in range(10):
        if wp_idx + i*3 <len(rte)-1:
            next_angle_list.append(get_angle(car,rte[wp_idx + i*3][0]))
    idx = 0
    while idx<len(next_angle_list)-2 and abs(next_angle_list[idx])>40:
        idx +=1
    return wp_idx+idx*3,next_angle_list[idx]

def get_distant_angle(car,wp_idx,rte, delta):
    if wp_idx + delta < len(rte)-1:
        i = wp_idx + delta
    else:
        i = len(rte)-1
    intersection_detected = False
    for x in range(i-wp_idx):
        if rte[wp_idx+x][0].is_junction:
             intersection_detected = True
    angle = get_angle(car,rte[i][0])
    if not intersection_detected:
        result = 0
    elif angle <-10:
        result = -1
    elif angle>10:
        result =1
    else:
        result = 0
    return result

def draw_route(wp, route,seconds=3.0):
        draw_colour = carla.Color(r=255, g=0, b=0)
    else:
        draw_colour = carla.Color(r=0, g=0, b=255)
    for i in range(10):
        if wp+i<len(route)-2:
            world.debug.draw_string(route[wp+i][0].transform.location, '^', draw_shadow=False,
                color=draw_colour, life_time=seconds,
                persistent_lines=True)
    return None

def select_random_route(position,locs):
    point_a = position.location #we start at where the car is or last waypoint
    sampling_resolution = 1
    grp = GlobalRoutePlanner(world.get_map(), sampling_resolution)
    # now let' pick the longest possible route
    min_distance = 100
    result_route = None
    route_list = []
    for loc in locs: # we start trying all spawn points
                                #but we just exclude first at zero index
        cur_route = grp.trace_route(point_a, loc.location)
        if len(cur_route) > min_distance:
            route_list.append(cur_route)
    result_route = random.choice(route_list)
    return result_route


def exit_clean():
    for sensor in world.get_actors().filter('*sensor*'):
        sensor.destroy()
    for actor in world.get_actors().filter('*vehicle*'):
        actor.destroy()
    return None


pcd = None

def lidar_callback(point_cloud, point_list):
    """Prepares a point cloud with intensity
    colors ready to be consumed by Open3D"""
    data = np.copy(np.frombuffer(point_cloud.raw_data, dtype=np.dtype('f4')))
    data = np.reshape(data, (int(data.shape[0] / 4), 4))
    points = data[:, :-1]
    points[:, :1] = -points[:, :1]

    pcd = points


def generate_lidar_bp(blueprint_library):
    """Generates a CARLA blueprint based on the script parameters"""
    lidar_bp = blueprint_library.find('sensor.lidar.ray_cast')

    lidar_bp.set_attribute('upper_fov', str(10.0))
    lidar_bp.set_attribute('lower_fov', str(-30.0))
    lidar_bp.set_attribute('channels', str(64.0))
    lidar_bp.set_attribute('range', str(100.0))
    lidar_bp.set_attribute('rotation_frequency', str(20))
    lidar_bp.set_attribute('points_per_second', str(1000000))
    return lidar_bp


def convert_lidar_to_pgm_polar(lidar_data, num_layers=64, num_angular_steps=360, max_range=100.0):
    x, y, z = lidar_data[:, 0], lidar_data[:, 1], lidar_data[:, 2]
    r = np.sqrt(x**2 + y**2)  # Radial distance
    theta = np.arctan2(y, x)  # Angle in radians
    theta_degrees = np.degrees(theta)  # Convert to degrees
    theta_degrees = (theta_degrees + 360) % 360  # Ensure range [0, 360)

    # Discretize theta into bins
    angular_step_size = 360.0 / num_angular_steps
    theta_bins = (theta_degrees / angular_step_size).astype(int)

    # Normalize z values into layer indices
    layer_indices = ((z - np.min(z)) / (np.max(z) - np.min(z)) * (num_layers - 1)).astype(int)

    # Initialize a 2D grid (n x m) for PGM image
    pgm_grid = np.full((num_layers, num_angular_steps), 0, dtype=np.uint8)

    # Populate the PGM grid
    for i in range(len(lidar_data)):
        layer = layer_indices[i]
        angle_bin = theta_bins[i]
        distance = min(r[i], max_range)  # Clip the distance to max_range
        normalized_distance = int((distance / max_range) * 255)  # Normalize to [0, 255]
        pgm_grid[layer, angle_bin] = max(pgm_grid[layer, angle_bin], normalized_distance)

    return pgm_grid


def generate_traffic(world, client, num_vehicles=50, num_pedestrians=10):

    vehicles_list = []
    walkers_list = []
    all_id = []

    # Setup Traffic Manager
    traffic_manager = client.get_trafficmanager(8000)
    traffic_manager.set_global_distance_to_leading_vehicle(2.5)
    traffic_manager.set_synchronous_mode(True)
    world_settings = world.get_settings()
    world_settings.synchronous_mode = True
    world.apply_settings(world_settings)

    # Get blueprints
    vehicle_blueprints = world.get_blueprint_library().filter('vehicle.*')
    walker_blueprints = world.get_blueprint_library().filter('walker.pedestrian.*')

    # Spawn vehicles
    spawn_points = world.get_map().get_spawn_points()
    random.shuffle(spawn_points)
    batch = []

    for n, transform in enumerate(spawn_points[:num_vehicles]):
        blueprint = random.choice(vehicle_blueprints)
        if blueprint.has_attribute('color'):
            color = random.choice(blueprint.get_attribute('color').recommended_values)
            blueprint.set_attribute('color', color)
        blueprint.set_attribute('role_name', 'autopilot')
        batch.append(carla.command.SpawnActor(blueprint, transform)
                        .then(carla.command.SetAutopilot(carla.command.FutureActor, True, traffic_manager.get_port())))

    for response in client.apply_batch_sync(batch, True):
        if not response.error:
            vehicles_list.append(response.actor_id)

    # Spawn pedestrians
    walker_spawn_points = []
    for _ in range(num_pedestrians):
        spawn_point = carla.Transform()
        loc = world.get_random_location_from_navigation()
        if loc:
            spawn_point.location = loc
            walker_spawn_points.append(spawn_point)

    walker_batch = []
    walker_speeds = []

    for spawn_point in walker_spawn_points:
        walker_bp = random.choice(walker_blueprints)
        walker_bp.set_attribute('is_invincible', 'false')
        walker_batch.append(carla.command.SpawnActor(walker_bp, spawn_point))

    walker_results = client.apply_batch_sync(walker_batch, True)
    for result in walker_results:
        if not result.error:
            walkers_list.append({"id": result.actor_id})

    # Add walker controllers
    walker_controller_bp = world.get_blueprint_library().find('controller.ai.walker')
    controller_batch = []
    for walker in walkers_list:
        controller_batch.append(carla.command.SpawnActor(walker_controller_bp, carla.Transform(), walker["id"]))

    controller_results = client.apply_batch_sync(controller_batch, True)
    for i, result in enumerate(controller_results):
        if not result.error:
            walkers_list[i]["con"] = result.actor_id
            all_id.append(walkers_list[i]["con"])
            all_id.append(walkers_list[i]["id"])

    all_actors = world.get_actors(all_id)

    # Start the walkers
    for i in range(0, len(all_id), 2):
        all_actors[i].start()
        all_actors[i].go_to_location(world.get_random_location_from_navigation())
        all_actors[i].set_max_speed(1 + random.random())  # Random speed for walkers

    print(f"Spawned {len(vehicles_list)} vehicles and {len(walkers_list)} pedestrians.")


class ObstacleDetectorModel(nn.Module):
    def __init__(self):
        super(ObstacleDetectorModel, self).__init__()

        # Convolutional layers for PGM input
        self.pgm_branch = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),  # Assuming PGM is grayscale (1 channel)
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten()
        )

        # Convolutional layers for segmentation input
        self.seg_branch = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # Assuming segmentation is RGB (3 channels)
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten()
        )

        # Fully connected layers after concatenation
        self.fc = nn.Sequential(
            nn.Linear(128 * ((HEIGHT // 8) * (WIDTH // 8)) * 2, 128),  # Adjust based on the final size of the flattened layers
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 1),
            nn.Sigmoid()  # Output a probability for obstacle presence
        )

    def forward(self, pgm_input, seg_input):
        pgm_features = self.pgm_branch(pgm_input)
        seg_features = self.seg_branch(seg_input)

        # Concatenate the features from both branches
        combined_features = torch.cat((pgm_features, seg_features), dim=1)

        # Pass through fully connected layers
        output = self.fc(combined_features)
        return output


class AutoNaviModel(nn.Module):
    def __init__(self):
        super(AutoNaviModel, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64 * (HEIGHT // 8) * (WIDTH // 8), 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 4),
            nn.ReLU(),
            nn.Flatten()
        )
        self.output_layer = nn.Linear(5, 1)  # 4 from image + 1 from input_2

    def forward(self, image, navigational_direction):
        image_features = self.conv_layers(image)
        combined = torch.cat((image_features, navigational_direction.unsqueeze(1)), dim=1)
        output = self.output_layer(combined)
        return output


def control_car(sem_image, pgm_image, navigational_direction):
    transform = transforms.Compose([
        transforms.Resize((HEIGHT, WIDTH)),
        transforms.ToTensor(),
    ])
    sem_image = transform(sem_image).unsqueeze(0).to(DEVICE)
    pgm_image = transform(pgm_image).unsqueeze(0).to(DEVICE)

    with torch.no_grad():
        auto_navi_output = auto_navi_model(sem_image, navigational_direction)
        obsta_output = obsta_model(pgm_image, sem_image)

    return float(auto_navi_output), round(obsta_output)


auto_navi_model_path = '/content/drive/MyDrive/Thesis/Self-Driving-Car/Auto-Navi/models/auto_navi_model.pth'
obsta_model_path = '/content/drive/MyDrive/Thesis/Self-Driving-Car/Auto-Navi/models/obsta_model.pth'

auto_navi_model = torch.load(auto_navi_model_path, map_location=torch.device(DEVICE))
obsta_model = torch.load(obsta_model_path,map_location=torch.device(DEVICE))

auto_navi_model.eval()
obsta_model.eval()


In [ ]:
#main loop
client = carla.Client('localhost', 2000)
world = client.get_world()
spawn_points = world.get_map().get_spawn_points()

vehicle_bp = world.get_blueprint_library().filter('*model3*')[0]
quit = False
image_w = 640
image_h = 480

while True:
    start_point = random.choice(spawn_points)
    vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)
    time.sleep(2)

    blueprint_library = world.get_blueprint_library()

    camera_bp = blueprint_library.find('sensor.camera.semantic_segmentation')
    camera_bp.set_attribute('fov', '90')
    camera_bp.set_attribute('image_size_x', f'{image_w}')
    camera_bp.set_attribute('image_size_y', f'{image_h}')
    camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
    camera_sem = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)
    camera_data = {'sem_image': np.zeros((image_h,image_w,4))}
    camera_sem.listen(lambda image: sem_callback(image,camera_data))


    lidar_bp = generate_lidar_bp(blueprint_library)
    lidar_transform = carla.Transform(carla.Location(x=0, z=2))
    lidar = world.spawn_actor(lidar_bp, lidar_transform, attach_to=vehicle)
    point_list = o3d.geometry.PointCloud()
    lidar.listen(lambda data: lidar_callback(data, point_list))

    route = select_random_route(start_point,spawn_points)
    curr_wp = 5
    predicted_angle = 0
    PREFERRED_SPEED = 40


    while curr_wp<len(route)-1:
        world.tick()
        draw_route(curr_wp, route,1)
        if cv2.waitKey(1) == ord('q'):
            quit = True
            exit_clean()
            break

        sem_image = camera_data['sem_image']
        sem_image = cv2.resize(sem_image, (WIDTH,HEIGHT))

        if curr_wp >=len(route)-10:
            PREFERRED_SPEED = 0
            exit_clean()
            break

        while curr_wp<len(route)-2 and vehicle.get_transform().location.distance(route[curr_wp][0].transform.location)<5:
            curr_wp +=1 #move to next wp if we are too close
        curr_wp, predicted_angle = get_proper_angle(vehicle,curr_wp,route)
        distant_angle = get_distant_angle(vehicle,curr_wp,route,30)

        v = vehicle.get_velocity()
        speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2),0)

        estimated_throttle = maintain_speed(speed)

        pgm_image = convert_lidar_to_pgm_polar(pcd)

        steer_input, obstacle_detection = control_car(sem_image, pgm_image, distant_angle)

        if obstacle_detection == 0:
            vehicle.apply_control(carla.VehicleControl(throttle=float(estimated_throttle), steer=steer_input))
        elif obstacle_detection == 1:
            vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=1.0))

    if quit:
        break

